In [117]:
# imports
import pandas as pd
import random

import os

In [118]:
# read in decklist
# WE ARE RUNNING 21 SWAMPS

decklist = pd.read_csv(os.path.join('data', 'gonti,_night_minister.csv'))

sub_ndxs = [0,1,2,3]

decklist.iloc[sub_ndxs,:]

,Unnamed: 0,name,mana_cost,cmc,colors,type_line,oracle_text,produced_mana
0,77,"Shizo, Death's Storehouse",NaN,0.0,[],Legendary Land,"{T}: Add {B}.\r\n{B}, {T}: Target legendary cr...",['B']
1,497,Diabolic Intent,{1}{B},2.0,['B'],Sorcery,"As an additional cost to cast this spell, sacr...",NaN
2,533,Misinformation,{B},1.0,['B'],Instant,Put up to three target cards from an opponent'...,NaN
3,610,Deadly Rollick,{3}{B},4.0,['B'],Instant,"If you control a commander, you may cast this ...",NaN


In [119]:
# reading in text files
# as a heads up you need to delete the last line of this as its 

def read_decklist_txt(path) -> pd.DataFrame:
    with open(path, 'r') as f:
        string = f.read()
        string = string.split('\n')
        # first index is number
        numbers = [i.split(' ')[0] for i in string]

        # read categories

        cats = [i[i.find('['):i.find(']')+1].strip('[]')
                for i in string]
        cats = [i.split(',') for i in cats]

        # even out number of categories
        max_cat_len = max([len(c) for c in cats])
        cats = [i + (['NA'] * (max_cat_len-len(i)))
                for i in cats]

        # read color tags

        tags = [i[i.find('^'):-1].strip('^') for i in string]
        tags = [i.split(',')[0] for i in tags] # drop colors

        # get names
        names = [i[i.find(' ')+1:i.find('[')-1] for i in string]

        columns = sum(
            [['name'], [f'cat_{ndx}' for ndx in range(max_cat_len)],
             ['tags'], ['no']], []
        )

        decklist = pd.DataFrame(
            [sum([[names[ndx]], cats[ndx], [tags[ndx]], [numbers[ndx]]], [])
             for ndx in range(len(string))],
             columns=columns
        )

        return decklist

In [120]:
def get_decklist_info(card_data:pd.DataFrame, decklist:pd.DataFrame):
    """Map card data to a decklist"""
    return card_data[card_data['name'].isin(decklist['name'])]

decklist = read_decklist_txt(os.path.join('data', 'narser.txt'))
decklist.index.name = 'decklist_ndx'

# read in card_data from json
card_path = os.path.join('data', 'oracle-cards-20250414210533.json')
card_data = pd.read_json(card_path)

card_data.shape

(34504, 82)

In [121]:
decklist

,name,cat_0,cat_1,cat_2,cat_3,tags,no
decklist_ndx,,,,,,,
0,Access Tunnel,Evasion,NA,NA,NA,,1x
1,Adarkar Wastes,Land,NA,NA,NA,,1x
2,Aggravated Assault,Extra Combat,Extra Triggers,NA,NA,,1x
3,All That Glitters,Pump,NA,NA,NA,,1x
4,Alrund's Epiphany,Extra Turn,Extra Triggers,NA,NA,,1x
...,...,...,...,...,...,...,...
203,Temporal Manipulation,Extra Turn,Extra Triggers,NA,NA,,1x
204,Temporal Trespass,Extra Turn,Extra Triggers,NA,NA,,1x
205,Time Stretch,Extra Turn,Extra Triggers,NA,NA,,1x


In [122]:
# label indices
card_data.index.names = ['scryfall_ndx']

# match global card data to decklist
card_data = get_decklist_info(card_data, decklist).sort_values('name')

card_data = card_data[card_data.duplicated(subset=['name']) != True]

# make multidex for merging
card_data.index = pd.MultiIndex.from_tuples(
    [(card_data.index[i], i) 
     for i in range(card_data.shape[0])],
    names=['scryfall_ndx','decklist_ndx']
)

card_data.shape, decklist.shape

((208, 82), (208, 7))

In [123]:
# join decklist and card_data
# # add card_data indices to decklist

decklist.index = card_data.index

card_data = pd.concat([card_data, decklist], axis=1)

card_data.head()

,,object,id,oracle_id,multiverse_ids,mtgo_id,tcgplayer_id,cardmarket_id,name,lang,released_at,...,color_indicator,content_warning,defense,name,cat_0,cat_1,cat_2,cat_3,tags,no
scryfall_ndx,decklist_ndx,,,,,,,,,,,,,,,,,,,,,
32006,0,card,4bef5957-71a4-4fe0-b2ce-dff8e8690bd9,ed9cc560-f30b-4b60-a094-ccf93ed656a7,[696493],138615.0,624594.0,818907.0,Access Tunnel,en,2025-04-11,...,NaN,NaN,NaN,Access Tunnel,Evasion,NA,NA,NA,,1x
28761,1,card,1f056e21-63b3-4684-be5b-04ac220c4a52,d5ad26cc-2bdb-46b7-b8bf-dd099d5fa09b,[696494],138617.0,624202.0,818908.0,Adarkar Wastes,en,2025-04-11,...,NaN,NaN,NaN,Adarkar Wastes,Land,NA,NA,NA,,1x
4347,2,card,c99c5707-d5f2-4675-bfca-e801e6b0f627,20129459-a386-41eb-899d-1aede3427300,[40195],18101.0,10348.0,1816.0,Aggravated Assault,en,2002-10-07,...,NaN,NaN,NaN,Aggravated Assault,Extra Combat,Extra Triggers,NA,NA,,1x
22113,3,card,5fc0b82a-f943-4330-b9e7-bb4527354bfd,a4d751e0-41c1-4e90-853d-512f385acd81,[622617],115117.0,507115.0,723483.0,All That Glitters,en,2023-08-04,...,NaN,NaN,NaN,All That Glitters,Pump,NA,NA,NA,,1x
25267,4,card,c94fcb53-a7bd-4a80-a536-9fb0eb24261a,bc842838-1993-4480-b8fe-f9d9f2dee7a4,[503648],87407.0,230097.0,529482.0,Alrund's Epiphany,en,2021-02-05,...,NaN,NaN,NaN,Alrund's Epiphany,Extra Turn,Extra Triggers,NA,NA,,1x


In [124]:
def sample_cardlist(cardlist:pd.DataFrame, decksize:int=100):
    return cardlist.iloc[random.sample(
        cardlist.index.get_level_values(1).tolist(), k=decksize
    )]

def shuffle(cardlist:pd.DataFrame):
    return cardlist.sample(cardlist.shape[0], replace=False)

def samplev2(cardlist:pd.DataFrame, draw_to:int=10, no_draws:int=10):

    # sample 100 cards

    deck = sample_cardlist(cardlist)

    # shuffle

    deck = shuffle(deck)


    samples = deck.iloc[
        sum([[random.randint(0, deck.shape[0]-1) for i in range(draw_to)]
        for j in range(no_draws)], [])
    ]

    

    samples.loc[:,'hand_no'] = sum([[i] * draw_to for i in range(no_draws)], [])
    
    samples = samples.set_index('hand_no', append=True)

    return samples

card_data = samplev2(card_data, draw_to=11)

C:\Users\Alex\AppData\Local\Temp\ipykernel_7124\4247162986.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  samples.loc[:,'hand_no'] = sum([[i] * draw_to for i in range(no_draws)], [])
C:\Users\Alex\AppData\Local\Temp\ipykernel_7124\4247162986.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples.loc[:,'hand_no'] = sum([[i] * draw_to for i in range(no_draws)], [])


In [125]:
# make categories vertical



In [126]:

# NEED TO DO THIS FOR EACH category 
# card_data.groupby('hand_no').value_counts(['cat_0', 'cat_1', 'cat_2']) # can't group together

def get_archidekt_categories(card_data:pd.DataFrame, prefix:str='cat') -> list[str]:
    """pull out column names with a specified prefix"""
    categories = [i for i in card_data.columns.tolist() if prefix in i]
    categories = [i for i in categories if i.split('_')[0] == prefix]

    return categories


def get_unique_categories(card_data:pd.DataFrame) -> list[str]:

    # intialize series
    cats = pd.Series()

    # iterate through categories and concatanate verical axis
    for cat_i in get_archidekt_categories(card_data):
        # cats = pd.concat([cats, ])
        cats = pd.concat([cats, card_data[cat_i]])
    

    # return unique values 
    return pd.unique(cats).tolist()


In [127]:
# we can represent each string as a binary list based on the category order

# get the matching index of 

sums = []


for col in get_archidekt_categories(card_data):

    sums.append([
        (card_data[col].index.get_level_values(2).tolist()[ndx], ndx, i) 
        for ndx, i in enumerate(card_data[col].tolist())
    ])

sums = sum(sums, [])
sums


[(0, 0, 'Win Condition'),
 (0, 1, 'Evasion'),
 (0, 2, 'Ramp'),
 (0, 3, 'Extra Turn'),
 (0, 4, 'Extra Turn'),
 (0, 5, 'Land'),
 (0, 6, 'Extra Turn'),
 (0, 7, 'Haste Enabler'),
 (0, 8, 'Counter Spell'),
 (0, 9, 'Win Condition'),
 (0, 10, 'Pump'),
 (1, 11, 'Ramp'),
 (1, 12, 'Evasion'),
 (1, 13, 'Top Deck Manipulation'),
 (1, 14, 'Evasion'),
 (1, 15, 'Ramp'),
 (1, 16, 'Stax'),
 (1, 17, 'Win Condition'),
 (1, 18, 'Land'),
 (1, 19, 'Win Condition'),
 (1, 20, 'Counter Spell'),
 (1, 21, 'Win Condition'),
 (2, 22, 'Sorcery'),
 (2, 23, 'Top Deck Manipulation'),
 (2, 24, 'Land'),
 (2, 25, 'Win Condition'),
 (2, 26, 'Ramp'),
 (2, 27, 'Maybeboard{noDeck}{noPrice}'),
 (2, 28, 'Win Condition'),
 (2, 29, 'Spell Copy'),
 (2, 30, 'Removal'),
 (2, 31, 'Board Wipe'),
 (2, 32, 'Land'),
 (3, 33, 'Protection'),
 (3, 34, 'Stax'),
 (3, 35, 'Win Condition'),
 (3, 36, 'Stax'),
 (3, 37, 'Counter Spell'),
 (3, 38, 'Protection'),
 (3, 39, 'Haste Enabler'),
 (3, 40, 'Stax'),
 (3, 41, 'Top Deck Manipulation'),
 (3, 4

In [128]:
card_counts = []


for col in get_archidekt_categories(card_data):
    card_counts = sum([[card_data[col].value_counts().tolist(), card_data[col].value_counts().index.tolist()], card_counts],[])

card_counts

[[110],
 ['NA'],
 [108, 2],
 ['NA', 'Spell Copy'],
 [78, 9, 5, 5, 3, 3, 2, 2, 1, 1, 1],
 ['NA',
  'Removal',
  'Tokens',
  'Extra Triggers',
  'Protection',
  'Instant',
  'Artifact',
  'Tap Lands',
  'Ramp',
  'Enchantment',
  'Sac Land'],
 [14, 11, 10, 10, 7, 7, 6, 6, 6, 5, 4, 4, 4, 3, 3, 3, 3, 2, 1, 1],
 ['Land',
  'Stax',
  'Win Condition',
  'Ramp',
  'Counter Spell',
  'Board Wipe',
  'Maybeboard{noDeck}{noPrice}',
  'Top Deck Manipulation',
  'Protection',
  'Evasion',
  'Extra Turn',
  'Removal',
  'Sac Land',
  'Sorcery',
  'Haste Enabler',
  'Pump',
  'Draw',
  'Spell Copy',
  'Extra Combat',
  'Enchantment']]

In [129]:
card_data[get_archidekt_categories].groupby('hand_no').value_counts()

hand_no  cat_0          cat_1           cat_2  cat_3
0        Extra Turn     Extra Triggers  NA     NA       3
         Counter Spell  Ramp            NA     NA       1
         Evasion        NA              NA     NA       1
         Haste Enabler  NA              NA     NA       1
         Land           NA              NA     NA       1
                                                       ..
9        Protection     NA              NA     NA       1
         Pump           NA              NA     NA       1
         Removal        NA              NA     NA       1
         Sorcery        NA              NA     NA       1
         Win Condition  NA              NA     NA       1
Name: count, Length: 89, dtype: int64

In [ ]:
# get unique values from cat columns

# vertically stack cats

def get_unique_categories(card_data:pd.DataFrame) -> list[str]:

    # intialize series
    cats = pd.Series()

    # iterate through categories and concatanate verical axis
    for cat_i in get_archidekt_categories(card_data):
        # cats = pd.concat([cats, ])
        cats = pd.concat([cats, card_data[cat_i]])
    

    # return unique values 
    return pd.unique(cats).tolist()

unique_categories = get_unique_categories(card_data)

['Win Condition',
 'Evasion',
 'Ramp',
 'Extra Turn',
 'Land',
 'Haste Enabler',
 'Counter Spell',
 'Pump',
 'Top Deck Manipulation',
 'Stax',
 'Sorcery',
 'Maybeboard{noDeck}{noPrice}',
 'Spell Copy',
 'Removal',
 'Board Wipe',
 'Protection',
 'Sac Land',
 'Extra Combat',
 'Draw',
 'Enchantment',
 'Tokens',
 'NA',
 'Extra Triggers',
 'Instant',
 'Artifact',
 'Tap Lands']

In [131]:
unique_categories = get_unique_categories(card_data)

# make strings into integer vectors

# INTENDED FUNCTIONALITY

empty_vector = [[0 for i in range(len(unique_categories))]
                for j in range(len(unique_categories))]

category_mappings = {
    cat:ndx for (ndx, cat) in enumerate(unique_categories)
}

for ndx, cat in enumerate(unique_categories):
    empty_vector[ndx][category_mappings[cat]] = 1

category_mappings = {
    cat:empty_vector[ndx] for (ndx, cat) in enumerate(unique_categories)
}

# map categories

for col in get_archidekt_categories(card_data):
    card_data[col] = card_data[col].map(category_mappings)

    # we can use explode on categories 

card_data[get_archidekt_categories(card_data)]

,,,cat_0,cat_1,cat_2,cat_3
scryfall_ndx,decklist_ndx,hand_no,,,,
10821,160,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
32706,162,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
24097,197,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
22353,61,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17910,20,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
12536,24,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
27118,15,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
30394,148,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [134]:
# splitting things


split_step = len(unique_categories)

split_values = range(0, card_data['cat_0'].explode().shape[0], split_step)

[i for i in split_values]


[0,
 26,
 52,
 78,
 104,
 130,
 156,
 182,
 208,
 234,
 260,
 286,
 312,
 338,
 364,
 390,
 416,
 442,
 468,
 494,
 520,
 546,
 572,
 598,
 624,
 650,
 676,
 702,
 728,
 754,
 780,
 806,
 832,
 858,
 884,
 910,
 936,
 962,
 988,
 1014,
 1040,
 1066,
 1092,
 1118,
 1144,
 1170,
 1196,
 1222,
 1248,
 1274,
 1300,
 1326,
 1352,
 1378,
 1404,
 1430,
 1456,
 1482,
 1508,
 1534,
 1560,
 1586,
 1612,
 1638,
 1664,
 1690,
 1716,
 1742,
 1768,
 1794,
 1820,
 1846,
 1872,
 1898,
 1924,
 1950,
 1976,
 2002,
 2028,
 2054,
 2080,
 2106,
 2132,
 2158,
 2184,
 2210,
 2236,
 2262,
 2288,
 2314,
 2340,
 2366,
 2392,
 2418,
 2444,
 2470,
 2496,
 2522,
 2548,
 2574,
 2600,
 2626,
 2652,
 2678,
 2704,
 2730,
 2756,
 2782,
 2808,
 2834]

# generating samples

In [ ]:
# constants

no_deck_samples = 1
no_samples_per_deck = 1000 # we will eventually do this to stability
no_mulligans = 3 
no_cards_in_hand = 7
cards_drawn = 2

# get sampling pool

deck = shuffle(sample_decklist(decklist, repeats))

# sample cards by
# shuffling the deck and drawing 9 cards
#   where the first 7 are the initial hand 
#   and the last two are your first two draws

samples = [
    draw(shuffle(deck), no_cards_in_hand+cards_drawn-1) 
    for i in range(no_samples_per_deck*no_mulligans)
]

# generate sampling indices

sampling_multidex = sum(sum([
    [[(m, n, i) for i in range(no_mulligans)]
    for n in range(no_samples_per_deck)]
    for m in range(no_deck_samples)
], []), [])

# generate data column names

column_names = sum(
    [
        [f'card_no_{no}' for no in range(no_cards_in_hand)],
        [f'draw_no_{no}' for no in range(cards_drawn)]
    ], []
)

data = pd.DataFrame(
    samples, 
    index=pd.MultiIndex.from_tuples(
        sampling_multidex, names=['deck_sample', 'deck_sample_no','mulligan_count']
    ),
    columns=column_names
)

# data.to_parquet(os.path.join(
#     'data', 'samples.parquet'
# ))

data

NameError: name 'sample_decklist' is not defined

In [ ]:
pd.read_parquet(os.path.join('data','samples.parquet'))

card_no_0  card_no_1  card_no_2  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         48         47   
                           1                      36         38         36   
                           2                      15         10         36   
            1              0                       4         35         80   
                           1                      21        105         67   
...                                              ...        ...        ...   
            998            1                      69         35         97   
                           2                      99         35         36   
            999            0                      51        103         97   
                           1                      60         43         13   
                           2                      96         36        106   

                                           card_no_3  card_no_4  card_no_5  \
deck_sample deck_sample_no mulligan_count                                    
0           0              0                      36         60         83   
                           1                      28         36         68   
                           2                      80         36         73   
            1              0                      33         36        113   
                           1                      36         74         81   
...                                              ...        ...        ...   
            998            1                      36         89         41   
                           2                     102         13         36   
            999            0                      36         36         36   
                           1                     113         47         32   
                           2                      36         29         33   

                                           card_no_6  draw_no_0  draw_no_1  
deck_sample deck_sample_no mulligan_count                                   
0           0              0                     105         36         59  
                           1                      37         59         29  
                           2                      61          7          4  
            1              0                      36         36        106  
                           1                     113         36        114  
...                                              ...        ...        ...  
            998            1                      36         36         80  
                           2                      43         21         36  
            999            0                      36          6         74  
                           1                       9         97         36  
                           2                      81         47         16  

[3000 rows x 9 columns]

In [ ]:
# need to add a way to map card types or whatever to the integers

def get_mappings(decklist:pd.DataFrame, column:str) -> dict:

    return decklist.loc[:,column].to_dict()


for col in data.columns:
    data[col] = data[col].map(get_mappings(decklist, 'type_line'))

data

card_no_0  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                                         Enchantment   
                           1                         Creature — Phyrexian Horror   
                           2                                   Creature — Spirit   
            1              0                                             Instant   
                           1                                             Sorcery   
...                                                                          ...   
            998            1                                             Instant   
                           2                                   Creature — Spirit   
            999            0                                                Land   
                           1                        Artifact — Equipment // Land   
                           2               Legendary Creature — Aetherborn Rogue   

                                                         card_no_1  \
deck_sample deck_sample_no mulligan_count                            
0           0              0                           Enchantment   
                           1                    Basic Land — Swamp   
                           2               Creature — Dauthi Rogue   
            1              0               Creature — Human Wizard   
                           1                  Artifact — Equipment   
...                                                            ...   
            998            1                                  Land   
                           2                           Enchantment   
            999            0                           Enchantment   
                           1                           Enchantment   
                           2                              Artifact   

                                                      card_no_2  \
deck_sample deck_sample_no mulligan_count                         
0           0              0               Artifact — Equipment   
                           1                        Enchantment   
                           2                            Instant   
            1              0                  Creature — Spirit   
                           1                 Basic Land — Swamp   
...                                                         ...   
            998            1                        Land — Gate   
                           2                        Enchantment   
            999            0                               Land   
                           1                        Enchantment   
                           2                        Enchantment   

                                                      card_no_3  \
deck_sample deck_sample_no mulligan_count                         
0           0              0               Artifact — Equipment   
                           1                            Sorcery   
                           2                        Enchantment   
            1              0                 Enchantment — Aura   
                           1                 Enchantment — Aura   
...                                                         ...   
            998            1               Artifact — Equipment   
                           2                            Instant   
            999            0                            Instant   
                           1                        Enchantment   
                           2                        Enchantment   

                                                                       card_no_4  \
deck_sample deck_sample_no mulligan_count                                          
0           0              0                            Creature — Dauthi Minion   
                           1                                   Creature

In [ ]:
# label card types

# lands


for col in data.columns:
    data.loc[data[col].str.find('Land') != -1, col] = 'land'
    data.loc[data[col].str.find('Creature') != -1, col] = 'creature'
    data.loc[data[col].str.find('Enchantment') != -1, col] = 'enchantment'
    data.loc[data[col].str.find('Equipment') != -1, col] = 'equipment' 
    data.loc[data[col].str.find('Sorcery') != -1, col] = 'sorcery'
    data.loc[data[col].str.find('Instant') != -1, col] = 'instant'
    data.loc[data[col].str.find('Artifact') != -1, col] = 'artifact'

# 

In [ ]:
# sum number of lands

categories = []

for col in data.columns:
    categories = sum(
        [[i for i in pd.unique(data[col]).tolist() 
         if i not in categories],categories],[]
    )

categories

['enchantment',
 'creature',
 'instant',
 'sorcery',
 'land',
 'artifact',
 'equipment']